#### NLP- EXPERIMENT 4
name: Gunj Vaviya

Roll No.: I073

Batch: B.Tech A.I.

Date: 26.08.25

In [ ]:
import nltk
from nltk.corpus import gutenberg
from nltk.util import ngrams
from collections import Counter

In [ ]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
hamlet_text = gutenberg.raw('shakespeare-hamlet.txt')

In [ ]:
hamlet_tokens = gutenberg.words('shakespeare-hamlet.txt')

In [ ]:
tokens_lower = [token.lower() for token in hamlet_tokens]

In [ ]:
print("First 20 tokens of Hamlet:", hamlet_tokens[:20])

First 20 tokens of Hamlet: ['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', 'William', 'Shakespeare', '1599', ']', 'Actus', 'Primus', '.', 'Scoena', 'Prima', '.', 'Enter', 'Barnardo', 'and', 'Francisco']


###TASK 1:

 a.	Write a python program to find all unigrams, bigrams and trigrams present in the given corpus  (Student may use any corpus specified in Lab 3).

In [ ]:
#UNIGRAMS:
hamlet_unigrams = list(ngrams(hamlet_tokens, 1))
print("\nFirst 10 unigrams of Hamlet:")
print(hamlet_unigrams[:10])


First 10 unigrams of Hamlet:
[('[',), ('The',), ('Tragedie',), ('of',), ('Hamlet',), ('by',), ('William',), ('Shakespeare',), ('1599',), (']',)]


In [ ]:
#BIGRAMS:
hamlet_bigrams = list(ngrams(hamlet_tokens, 2))
print("\nFirst 10 bigrams of Hamlet:")
print(hamlet_bigrams[:10])


First 10 bigrams of Hamlet:
[('[', 'The'), ('The', 'Tragedie'), ('Tragedie', 'of'), ('of', 'Hamlet'), ('Hamlet', 'by'), ('by', 'William'), ('William', 'Shakespeare'), ('Shakespeare', '1599'), ('1599', ']'), (']', 'Actus')]


In [ ]:
#TRIGRAMS:
hamlet_trigrams = list(ngrams(hamlet_tokens, 3))
print("\nFirst 10 trigrams of Hamlet:")
print(hamlet_trigrams[:10])


First 10 trigrams of Hamlet:
[('[', 'The', 'Tragedie'), ('The', 'Tragedie', 'of'), ('Tragedie', 'of', 'Hamlet'), ('of', 'Hamlet', 'by'), ('Hamlet', 'by', 'William'), ('by', 'William', 'Shakespeare'), ('William', 'Shakespeare', '1599'), ('Shakespeare', '1599', ']'), ('1599', ']', 'Actus'), (']', 'Actus', 'Primus')]


c.	Write a python program to predict the next word based on bigram model and trigram model

In [ ]:
def build_model(ngrams_list):
  model = {}
  for gram in ngrams_list:
    context = gram[:-1]
    next_word = gram[-1]
    if context not in model:
      model[context] = Counter()
      # Increment the count for the next word in the given context
      model[context][next_word] += 1
  return model

In [ ]:
def predict_next_word(model, context_tuple):
    if context_tuple in model:
        # Return the most common word in the Counter for this context
        return model[context_tuple].most_common(1)[0][0]
    else:
        return "No prediction available for this context."

In [ ]:
# Build the bigram model
bigram_model = build_model(hamlet_bigrams)

In [ ]:
# Demonstrate bigram prediction
print("\n--- Bigram Model Prediction ---")
bigram_context = ('of',)
predicted_word_bigram = predict_next_word(bigram_model, bigram_context)
print(f"Context: {bigram_context}")
print(f"Predicted next word: '{predicted_word_bigram}'")
print(f"(Based on the most common word following '{bigram_context[0]}')")


--- Bigram Model Prediction ---
Context: ('of',)
Predicted next word: 'Hamlet'
(Based on the most common word following 'of')

--- Bigram Model Prediction ---
Context: ('of',)
Predicted next word: 'Hamlet'
(Based on the most common word following 'of')


b.	Write a python program to find the probability of the given statement using bigrams

“This is my cat”
 by taking the following training data into consideration.
'This is a dog’,
'This is a cat',
 'I love my cat',
 'This is my name’


In [15]:
import nltk
from nltk import bigrams
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
# Training sentences
sentences = [
    "This is a dog",
    "This is a cat",
    "I love my cat",
    "This is my name"
]

In [18]:
test_sentence = "This is my cat"

In [19]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [20]:
def preprocess(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    return ["<s>"] + tokens + ["</s>"]

In [21]:
# Tokenize training sentences and generate bigrams
train_tokens = [preprocess(sent) for sent in sentences]

In [22]:
# Flatten the list of tokens for unigram counts
all_words = [word for sent in train_tokens for word in sent]

In [23]:
# Create Conditional Frequency Distribution for bigrams
cfd = ConditionalFreqDist()

for sent in train_tokens:
    for w1, w2 in bigrams(sent):
        cfd[w1][w2] += 1

In [24]:
# Create Conditional Probability Distribution using MLE
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [25]:
# Preprocess the test sentence
test_tokens = preprocess(test_sentence)

In [26]:
# Calculate bigram probability of the test sentence
prob = 1.0
for w1, w2 in bigrams(test_tokens):
    if w1 in cpd:
        prob_w2_given_w1 = cpd[w1].prob(w2)
        # If bigram probability is zero (not seen), overall prob is zero
        if prob_w2_given_w1 == 0:
            prob = 0
            break
        prob *= prob_w2_given_w1
    else:
        prob = 0
        break

print(f"Probability of the sentence '{test_sentence}' is: {prob}")

Probability of the sentence 'This is my cat' is: 0.125


d.	Write a python program to calculate the perplexity of a test text based on n-gram probabilities

In [31]:
def preprocess(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    return ["<s>"] + tokens + ["</s>"]

In [36]:
import math
from nltk.util import bigrams

def calculate_perplexity(sentence, cpd_model):
    """
    Calculates the perplexity of a sentence given a bigram probability model.

    Args:
        sentence (str): The input sentence.
        cpd_model (ConditionalProbDist): The bigram probability model (ConditionalProbDist).

    Returns:
        float: The perplexity of the sentence.
    """
    # Preprocess the sentence (add <s> and </s>)
    tokens = preprocess(sentence)
    n = len(tokens) - 1 # Number of bigrams

    # Calculate the probability of the sentence using the bigram model
    sentence_prob = 1.0
    for w1, w2 in bigrams(tokens):
        if w1 in cpd_model and cpd_model[w1].prob(w2) > 0:
            sentence_prob *= cpd_model[w1].prob(w2)
        else:
            # Handle unseen bigrams (assign a very small probability or use smoothing)
            # For simplicity here, we'll assign a small probability
            sentence_prob *= 1e-10 # Assign a small probability for unseen bigrams

    # Calculate perplexity
    if sentence_prob == 0:
        return float('inf')  # Perplexity is infinite if probability is zero
    perplexity = (1/sentence_prob)**(1/n)
    return perplexity

In [37]:
# Calculate and print the perplexity of the test sentence
perplexity_score = calculate_perplexity(test_sentence, cpd)
print(f"Perplexity of the sentence is: {perplexity_score:.4f}")

Perplexity of the sentence is: 1.5157


######Conclusion:
These three lines can be thought of as gathering the essential components for your task. import nltk brings in a massive toolkit designed for working with human language, giving you access to pre-built functions for tasks like tokenizing a sentence into words. From that toolkit, from nltk.util import ngrams selects a specific, powerful tool that is designed to group those words into sequences, like the two-word pairs needed for a bigram model. Finally, from collections import Counter provides a highly efficient tool that acts as an automatic tally sheet, quickly counting how many times each word or word pair appears in your training data, which is a fundamental step in calculating probabilities.